In [1]:
import numpy as np
import quaternion
import torch as T
import mujoco

from gymnasium.spaces import Space

from unitree_robot.train.environments import Go2Env, MujocoEnv
from unitree_robot.train.training import Trainer
from unitree_robot.train.experiments import StandUpExperiment

In [2]:
MJCF_PATH = "./external/unitree_mj_models/go2/scene.xml"
SEED = 0
DEVICE = "cpu"
BODY_ANGLE_REWARD_SCALE = 1.0
BODY_HEIGHT_REWARD_SCALE = 1.0
ENERGY_REWARD_SCALE = 1.0
JOINT_LIMIT_REWARD_SCALE = 1.0

SIM_FRAMES_PER_STEP = 5

NETWORK_HIDDEN_SIZE = 16
UNROLL_LENGTH = 256 # number of actions taken in the environment (inbetween these actions there are SIM_FRAMES_PER_STEP steps of simulation)
NUM_UNROLLS = 4 # number of full unrolls to collect training samples
MINIBATCH_SIZE = 32 # the sequence length that is trained on (UNROLL_LENGTH has to be divisible by this number)
TRAIN_BATCH_SIZE = 4 # the number of sequences that are used for a single training step (all batches are used in one epoch)

In [3]:
np.random.seed(seed=SEED)

experiment=StandUpExperiment(
    body_name = "base_link",
    body_angle_reward_scale = BODY_ANGLE_REWARD_SCALE,
    body_height_reward_scale = BODY_HEIGHT_REWARD_SCALE,
    energy_reward_scale = ENERGY_REWARD_SCALE,
    joint_limit_reward_scale = JOINT_LIMIT_REWARD_SCALE
)

env = Go2Env(
    model_path = MJCF_PATH,
    sim_frames_per_step = SIM_FRAMES_PER_STEP,
    experiment = experiment
)


trainer = Trainer(
    env=env,
    device=DEVICE,
    network_hidden_size=NETWORK_HIDDEN_SIZE,
    rng_seed=SEED
)

Trainer: device set to cpu !


In [4]:
trainer.train(
    unroll_length=UNROLL_LENGTH,
    num_unrolls=NUM_UNROLLS,
    minibatch_size=MINIBATCH_SIZE,
    train_batch_size=TRAIN_BATCH_SIZE
)

epoch: 0
torch.Size([4, 8, 32, 48])
torch.Size([4, 8, 32, 24])
torch.Size([4, 8, 32, 12])
torch.Size([4, 8, 32, 3])
batch: 0
torch.Size([4, 32, 12]) torch.Size([4, 32, 12]) torch.Size([4, 32, 12])


RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0

---

# Visualization

In [92]:
env.model.camera("main").pos = np.array([1, 1, 3])

try:
    i = 0
    while True:
        
        action = env.action_space.sample()
        env.do_simulation(ctrl=action, n_frames=5)

        env.render(cam_offset=np.array([1,1,5]))

        # print(env.data.qfrc_actuator)
       
        # print(r.as_euler("xyz", degrees=True))
        print(calc_angle(env.data.body("base").xquat))
        
        i += 1
        
except Exception as e:
    raise e
finally:
    env.close()

0.004169552843660834
0.008167382883955089
0.013082245474181322
0.01790481086854437
0.02514971524834672
0.03404143946854573
0.04008746493023648
0.04558147515398156
0.047372938099034555
0.04406269768778954
0.03904411185432921
0.031486527719814396
0.024042837807122498
0.02464562475312972
0.032534245588483536
0.041522138028459214
0.05020646426107461
0.052511693716083614
0.053440982796211275
0.05770918189109649
0.05792239602359791
0.05874453406295254
0.06152977305866081
0.05984662488630158
0.05498073509993302
0.04900967330886858
0.044139929093884325
0.04012202946105098
0.039553048369479246
0.040193972103887164
0.04128825459522146
0.04300131931670335
0.046418023996767796
0.04539089273062663
0.03958640054403312
0.032416391816599936
0.028182963721489523
0.01977326097893512
0.009624484701141404
0.003573963740660168
0.015965094853651298
0.026094862100124096
0.03462565010520858
0.04038917819082467
0.04403100784993748
0.04881736090487598
0.050056466569073206
0.05035169420145757
0.04987482871575711

C:\programming\unitree-robot\.venv-gym\lib\site-packages\glfw\__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


0.03457646967234209
0.03476180161781508
0.03555869851536534
0.03567819020070173
0.03396822841383927
0.03257943931293229
0.0346314997812931
0.037543815515336795
0.04138139843741138
0.04304443807190128
0.04404745142457135
0.04529089961104445
0.0454188602025832
0.04689063726444734
0.048544610940859094
0.05028731767485765
0.053444875741324296
0.05574105346390311
0.05911371869614356
0.06566730914854883
0.07256795129158328
0.0782740728512288
0.08109657772398368
0.08291652454581924
0.08443849587519783
0.08512280501238875
0.08522865380173707
0.0870001558837668
0.09081857983922426
0.09562595396574926
0.0955564801333126
0.09375596985597653
0.09096141630752846
0.08704596322943177
0.07837734781661942
0.06772320025897857
0.06539785611125386
0.06855931916809045
0.06761993649690333
0.061729432320403045
0.05522262098950027
0.04901073050247035
0.046131263614507596
0.04545776442661799
0.042590679884465917
0.041872765216027064
0.044964122215212114
0.04745940154336947
0.05120437735778866
0.054267580720184

KeyboardInterrupt: 

In [9]:
from unitree_robot.train.environments import lookat


target = env.data.cam("main").xpos + np.array([0, 0, -1])
target

# lookat(env.data.cam("main").xpos, target)


In [2]:
# env.action_space

# len(env.data.qpos)

# env.data.("imu_quat")


# q_rotate = quaternion.as_quat_array(env.data.sensor("imu_quat").data)
# quaternion.rotate_vectors(q_rotate, np.array([0,0,1]))
# (np.array([1, 0, 0]))




# env.get_sensor_state()

# env.data.xquat.shape